In [ ]:
import pyccl as ccl

# Define cosmology
cosmo = ccl.CosmologyVanillaLCDM()

# Define the galaxy and shape catalogues
galaxy_sample = {'RA': galaxies['RA'], 'DEC': galaxies['DEC'], 'Z': galaxies['Z'], 'W': galaxies['WEIGHT']}
shape_sample = {'RA': shapes['RA'], 'DEC': shapes['Dec'], 'E1': shapes['e1'], 'E2': shapes['e2'], 'W': shapes['w_iv']}

# Convert to comoving distances
galaxy_sample['D'] = ccl.comoving_radial_distance(cosmo, 1 / (1 + galaxy_sample['Z']))
shape_sample['D'] = ccl.comoving_radial_distance(cosmo, 1 / (1 + shapes['Z']))

# Calculate xi_{g+}(r_perp, r_par)
r_perp_bins = np.logspace(np.log10(0.1), np.log10(200), 20)
r_par_bins = np.linspace(-200, 200, 41)
xi_gp = ccl.correlation_3dRps(cosmo, galaxy_sample, shape_sample, r_perp_bins, r_par_bins)

# Plot the results
plt.figure(figsize=(10, 5))
plt.imshow(np.log10(np.abs(xi_gp)), extent=[r_perp_bins.min(), r_perp_bins.max(), r_par_bins.min(), r_par_bins.max()], aspect='auto')
plt.colorbar(label=r'$\xi_{g+}(r_\perp, r_\parallel)$')
plt.xlabel(r'$r_\perp$ [Mpc/h]')
plt.ylabel(r'$r_\parallel$ [Mpc/h]')
plt.xscale('log')
plt.title(r'$\xi_{g+}(r_\perp, r_\parallel)$')
plt.show()